In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
from monai.networks.nets import resnet10
import torch
import pandas as pd



c:\Users\098986\AppData\Local\anaconda3\envs\CILM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
mask = pd.read_table('../../masks.txt', header=None, sep=' ')
mask_1 = mask.apply(lambda x: x.str.split('.').str[0] + '_0000.nii.gz')
#mask = mask.apply(lambda x: x.str.split('_').str[0]+'_0000.nii.gz')
#write txt
mask_1.to_csv('../../../Data/HGC/Data_info/masks.txt', header=None, index=None, sep=' ')

In [2]:
torch.optim.Adam(resnet10().parameters(),lr=0.01)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [3]:
resnet = resnet10(spatial_dims=3,n_input_channels=1, num_classes=2)


In [4]:
random_input = torch.rand((1, 1, 256, 256,30))
resnet.train()

max_epochs = 100
optimizer = torch.optim.Adam(resnet.parameters(), 1e-5)
loss = torch.nn.CrossEntropyLoss()
label = torch.tensor([0])
for epoch in range(10):
    print(f"Epoch {epoch + 1}/{max_epochs}")
    for i in range(2):
        print(f"Batch {i + 1}/100")
        out_put_raw = resnet(random_input)
        optimizer.zero_grad()
        loss_value = loss(out_put_raw, label)
        

        loss_value.backward()
        optimizer.step()
        print(f"Loss: {loss_value.item()}")


Epoch 1/100
Batch 1/100


KeyboardInterrupt: 

In [5]:
def save_checkpoint(state_dict,save_path,file_name):
    """
    args:
        state_dict: model parameters
        save_path: file save path
        file_name: file save name
    """

    save_name = os.path.join(save_path,file_name)
    torch.save(state_dict,save_name)
    current_epoch = state_dict['epoch']
    print("save model to {}".format(save_name))
    print("current epoch is {}".format(current_epoch))

In [6]:
save_dict = {
    'epoch':1,
    'model':resnet.state_dict(),
    'optimizer':optimizer.state_dict(),
    'loss':loss.state_dict(),
    'arch':'resnet10'

}

save_checkpoint(save_dict,'./','test.pth')

save model to ./test.pth
current epoch is 1


In [78]:
model_dict = torch.load('./test.pth')['model']




In [84]:
resnet.load_state_dict(model_dict)
loss.load_state_dict(torch.load('./test.pth')['loss'])
optimizer.load_state_dict(torch.load('./test.pth')['optimizer'])
loss(resnet(random_input),label)

tensor(0.2212, grad_fn=<NllLossBackward0>)